In [ ]:
### kserve sdk로 torchserve 실행하기

### 다음의 과정을 이해해야 합니다.
* torchserve 실행을 위한 mar file생성
* minio를 local에 실행하기 위한 sa 설정
* marfile minIO local에 저장

### kserve sdk란?
* kserve sdk는 파이썬에서 inference 모델을 만드는 라이브러리
* yaml을 대신 만들어줌
* mar file, config.properties file을 불러오면 torchserve가 구현된 inferenceservice를 생성할 수 있음.


In [21]:
import json
x = {'output': [4]}

json.dumps(x)


'{"output": [4]}'

In [10]:
from kserve import (
    constants,
    KServeClient,
    V1beta1InferenceService,
    V1beta1InferenceServiceSpec,
    V1beta1PredictorSpec,
    V1beta1ModelSpec,
    V1beta1ModelFormat,
    V1beta1TorchServeSpec,
    utils
)
from kubernetes import client


In [15]:
client.V1ResourceRequirements(requests={'cpu':1,'memory':'1G'})

{'limits': None, 'requests': {'cpu': 1}}

In [4]:
service_name = 'pytorchserve2'
namespace = utils.get_default_target_namespace()
api_version = constants.KSERVE_GROUP + '/' + constants.KSERVE_V1BETA1_VERSION
storage = 's3://pytorch'

torchsvc= V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(name=service_name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                               spec=V1beta1InferenceServiceSpec(
                                   predictor=V1beta1PredictorSpec(
                                       service_account_name='sa',
                                       pytorch=(V1beta1TorchServeSpec(protocol_version='v1',resources=client.V1ResourceRequirements(requests={'cpu':1,'memory':'1G'}),
                                            storage_uri = storage))))
)

In [5]:
torchsvc

{'api_version': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
              'cluster_name': None,
              'creation_timestamp': None,
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': None,
              'name': 'pytorchserve2',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': None,
              'self_link': None,
              'uid': None},
 'spec': {'explainer': None,
          'predictor': {'active_deadline_seconds': None,
                        'affinity': None,
                        'automount_service_account_token': None,
                        'batcher': None,
                        'canary_traffic_percent': None,
     

In [ ]:
KServe = KServeClient()
KServe.create(torchsvc)
# torchsvc

In [ ]:
torch_resp=KServe.get(service_name, namespace=namespace)

In [ ]:
data = "Stopped back by Mellow Mushroom with my mate Justin from Brew Bros."
torch_resp['status']['address']['url']

In [ ]:
import requests
isvc_url = f'http://{service_name}.kubeflow-user-example-com.svc.cluster.local/v1/models/bert:predict'
print(isvc_url)

response = requests.post(isvc_url, json=data)
print(response.text)